# Test Bokeh chart interaction

## Imports

In [1]:
import pandas as pd
import random
import string
from bokeh.embed import json_item
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, BasicTickFormatter, BoxSelectTool, CustomJS, )
from bokeh.plotting import figure, show
from bokeh.transform import transform

## Prepare some data

In [2]:
letters = list(string.ascii_lowercase) + list(string.ascii_uppercase)
data = []
for i in range(0, 52):
    for j in range(0, 52):
        data.append([letters[i], f'{j}', random.randrange(100)])

chartdata = pd.DataFrame(columns=['x', 'y', 'z'], data=data)
chartdata

,x,y,z
0,a,0,61
1,a,1,18
2,a,2,70
3,a,3,68
4,a,4,54
...,...,...,...
2699,Z,47,25
2700,Z,48,21
2701,Z,49,38
2702,Z,50,23


## Plot the chart

In [15]:
source = ColumnDataSource(chartdata)
colors = ["#B21F35", "#D82735", "#FF7435", "#FFA135", "#FFCB35", "#FFF735", "#16DD36", "#009E47",
          "#00753A"]
mapper = LinearColorMapper(palette=colors, low=0, high=100)

TOOLTIPS = [
                ("x, y: ", "@y, @x"),
                ("Value: ", "@z"),
            ]
p = figure(title=f"Test",
           x_range=chartdata['x'].drop_duplicates(),
           y_range=chartdata['y'].drop_duplicates(),
           toolbar_location='below', tools=[BoxSelectTool(dimensions='width')],
           x_axis_location="above", tooltips=TOOLTIPS)


p.rect(x='x', y='y', width=1, height=1, source=source, line_color=None,
       fill_color=transform('z', mapper))

color_bar = ColorBar(color_mapper=mapper,
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=BasicTickFormatter())

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

# When a range is selected on the chart, the form from and to dates should be updated


source.selected.js_on_change(
    "indices",
    CustomJS(
        args=dict(source=source),
        code="""
            var inds = cb_obj.indices;
            var data = source.data;
            var xstart = data['x'][inds[0]]
            var xend = ''
            
            // Try getting +1
            xend = data['x'][inds[inds.length - 1] +1 ]
             
            // If undefined, get last
            if (typeof(xend) == "undefined") {
                xend = data['x'][inds[inds.length - 1]]
            }
            alert('From: ' + xstart + ' To: ' + xend)
            """,
    ),
)

show(p)

# A candlestick chart

## Imports

In [5]:
from math import pi

import pandas as pd

import bokeh

from bokeh.plotting import figure, output_file, show

## Download some sample data

In [6]:
bokeh.sampledata.download()
from bokeh.sampledata.stocks import MSFT

Creating C:\Users\jlcas\.bokeh directory
Creating C:\Users\jlcas\.bokeh\data directory
Using data directory: C:\Users\jlcas\.bokeh\data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3171836 bytes)
   3171836 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (4816256 bytes)
   4816256 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Do

## View the MSFT sample data

In [8]:
df = pd.DataFrame(MSFT)[:50]
df["date"] = pd.to_datetime(df["date"])
df

,date,open,high,low,close,volume,adj_close
0,2000-03-01,89.62,94.09,88.94,90.81,106889800,33.68
1,2000-03-02,91.81,95.37,91.12,93.37,106932600,34.63
2,2000-03-03,94.75,98.87,93.87,96.12,101435200,35.65
3,2000-03-06,96.00,97.37,90.12,90.62,93609400,33.61
4,2000-03-07,96.12,97.50,91.94,92.87,135061000,34.45
5,2000-03-08,93.81,96.19,91.00,95.56,94290000,35.44
6,2000-03-09,95.31,100.00,95.00,100.00,88198800,37.09
7,2000-03-10,99.56,102.50,99.50,101.00,85589000,37.46
8,2000-03-13,97.62,100.25,97.50,98.00,61831800,36.35
9,2000-03-14,98.62,99.25,95.12,95.12,73489200,35.28


# Plot the candlestick chart

In [9]:
inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = "MSFT Candlestick")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

show(p)